Problem 1

In [ ]:
%%sql
drop table if exists yttlj;

In [ ]:
%%sql
create table yttlj
(
    phaseId   int,
    phaseText text
);

In [ ]:
%%sql
-- WARNING: change the path to the file
copy yttlj from '/var/lib/postgresql/yttlj.csv' delimiter ',' csv header;

Problem 2

In [ ]:
%%sql
drop table if exists person;

In [ ]:
%%sql
create table person
(
    personId   int,
    personName text
);

In [8]:
%%sql
-- WARNING: change the path to the file
copy person from '/var/lib/postgresql/Person.csv' delimiter ',' csv header;

Problem 3

In [ ]:
%%sql
-- create table for coCurrent
drop table if exists coCurrent;

In [ ]:
%%sql
create table coCurrent
(
    personId1 int,
    personId2 int,
    counts    int
);

In [ ]:
%%sql
-- get bit string for each person
create or replace function getBitString(pName text) returns bit(45) as
$$
declare
    bitString bit(45) := repeat('0', 45)::bit(45);
    idx       int     := 0;
begin
    for idx in 0..44
        loop
            if exists (select 1 from yttlj where phaseId = idx and phaseText like '%' || pName || '%') then
                bitString := bitString | (1::bit(45) << idx);
            end if;
        end loop;
    return bitString;
end;
$$ language plpgsql;

In [ ]:
%%sql
alter table person add column phaseContains bit(45);

In [ ]:
%%sql
-- update person
update person
set phaseContains = getBitString(personName)
where person.personid < 1000; -- make sql happy

In [ ]:
%%sql
-- create function to count bits
create or replace function bitCount(bitString bit(45)) returns int as
$$
declare
    count int := 0;
    idx   int;
begin
    for idx in 0..44
        loop
            if (bitString & (1::bit(45) << idx)) <> 0::bit(45) then
                count := count + 1;
            end if;
        end loop;
    return count;
end;
$$ language plpgsql;

In [ ]:
%%sql
-- update coCurrent
insert into coCurrent
select p1.personId, p2.personId, bitCount(p1.phaseContains & p2.phaseContains)
from person p1
         join person p2 on p1.personId <= p2.personId;

In [32]:
%%sql
select * from cocurrent;

,personid1,personid2,counts
0,0,0,34
1,0,1,22
2,0,2,12
3,0,3,20
4,0,4,20
...,...,...,...
226,18,19,9
227,18,20,13
228,19,19,16
229,19,20,8


Problem 4 & 5

In [33]:
import numpy as np


def nmf_train(V, components, iternum, e):
    """
    非负矩阵分解函数
    :param V:  原始矩阵
    :param components:  要提取多少个特征
    :param iternum: 迭代次数
    :param e: 误差阈值
    :return:
    """
    m, n = V.shape
    # 随机初始化两个矩阵
    W = np.random.random((m, components))
    H = np.random.random((components, n))

    for iter in range(iternum):
        V_pre = np.dot(W, H)
        E = V - V_pre

        err = np.sum(E * E)
        if iter % 1000 == 0:
            print(err)
        if err < e:
            break
        # 对照更新公式
        a = np.dot(W.T, V)
        b = np.dot(W.T, np.dot(W, H))
        H[b != 0] = (H * a / b)[b != 0]

        c = np.dot(V, H.T)
        d = np.dot(W, np.dot(H, H.T))

        W[d != 0] = (W * c / d)[d != 0]
    return W, H

In [34]:
import psycopg2

# WARNING: change the database name, user and password
connection = psycopg2.connect("dbname=practice3 user=crl password=crlcrl123")
cursor = connection.cursor()

cursor.execute("select * from coCurrent")
connection.commit()
coCurrent = cursor.fetchall()
cursor.close()
connection.close()

np.random.seed(0)

max_id = 0
for row in coCurrent:
    max_id = max(max_id, row[0], row[1])

V = np.zeros((max_id + 1, max_id + 1))
for row in coCurrent:
    V[row[0], row[1]] = row[2]
V = V + V.T
for i in range(V.shape[0]):
    V[i, i] = V[i, i] / 2
print(V)

word_vectors, H = nmf_train(V, 50, 10000, 1e-4)

[[34. 22. 12. 20. 20. 22. 15. 27. 19. 15.  7.  9.  1. 27.  5.  3. 21. 10.
  21. 13. 13.]
 [22. 23. 12. 15. 16. 17. 10. 17. 12.  7.  4.  6.  0. 20.  5.  2. 12.  4.
  13.  8. 10.]
 [12. 12. 13.  9.  7.  8.  4.  9.  7.  2.  0.  3.  0. 12.  1.  0.  5.  1.
   5.  4.  5.]
 [20. 15.  9. 20. 16. 17.  8. 15. 12. 10.  4.  6.  0. 15.  1.  1. 10.  5.
  12.  5.  6.]
 [20. 16.  7. 16. 21. 19. 10. 18. 13. 10.  4.  6.  0. 17.  3.  2. 14.  6.
  15.  9.  8.]
 [22. 17.  8. 17. 19. 24. 13. 21. 18. 13.  6.  9.  0. 18.  4.  3. 17.  8.
  19.  9. 11.]
 [15. 10.  4.  8. 10. 13. 17. 17. 12.  7.  3.  6.  0. 14.  4.  3. 14.  3.
  14.  4. 11.]
 [27. 17.  9. 15. 18. 21. 17. 32. 24. 18.  9. 12.  1. 28.  6.  5. 26. 12.
  22. 12. 16.]
 [19. 12.  7. 12. 13. 18. 12. 24. 24. 16.  9. 11.  1. 21.  4.  3. 21. 11.
  17.  9. 12.]
 [15.  7.  2. 10. 10. 13.  7. 18. 16. 18.  9.  9.  1. 15.  2.  2. 17. 12.
  13.  7.  6.]
 [ 7.  4.  0.  4.  4.  6.  3.  9.  9.  9.  9.  6.  1.  9.  2.  1.  8.  7.
   7.  3.  4.]
 [ 9.  6.  3.  6.  6.

In [ ]:
%%sql
create extension if not exists vector;

In [79]:
%%sql
alter table person add column word_vector vector(50);

In [35]:
# WARNING: change the database name, user and password
connection = psycopg2.connect("dbname=practice3 user=crl password=crlcrl123")
cursor = connection.cursor()

for person_id, word_vector in enumerate(word_vectors):
    cursor.execute("update person set word_vector = '[{}]' where personid = {}"
                   .format(",".join(word_vector.astype(str)), person_id))

connection.commit()
cursor.close()
connection.close()

In [37]:
%%sql
select p1.personName, p2.personName, p1.word_vector <=> p2.word_vector as similarity
from person p1, person p2
where p1.personId < p2.personId
order by similarity 
limit 20;

,personname,personname.1,similarity
0,张无忌,杨逍,0.204253
1,俞莲舟,殷梨亭,0.226738
2,张三丰,俞莲舟,0.229455
3,赵敏,范遥,0.234185
4,张三丰,殷梨亭,0.253494
5,张三丰,谢逊,0.265402
6,张无忌,谢逊,0.280180
7,殷梨亭,周芷若,0.289047
8,殷梨亭,灭绝师太,0.289276
9,张无忌,灭绝师太,0.298040
